In [1]:
%matplotlib inline
import os
import shutil
import sys
import numpy as np
import pandas as pd
import mxnet as mx
from mxnet import gluon
from mxnet import image
from mxnet import nd
from mxnet import init
from mxnet import autograd
from mxnet.gluon.data import vision
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
from PIL import Image
from time import time
from tqdm import tqdm
import h5py

In [2]:
def get_features(net, data):
    features = []
    labels = []

    for X, y in tqdm(data):
        feature = net.features(X.as_in_context(ctx))
        features.append(feature.asnumpy())
        labels.append(y.asnumpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

In [3]:
def mkdir_if_not_exist(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))


def reorg_data(label_file, train_dir, test_dir, input_dir, valid_ratio):
    labels = pd.read_csv(label_file)
    labels = labels.sort_values('id')
    files = os.listdir(train_dir)
    files.sort()
    count = 0
    idx = np.arange(len(files))
    np.random.shuffle(idx)
    shutil.rmtree(os.path.join(input_dir, 'train'))
#     shutil.rmtree(os.path.join(input_dir, 'valid'))
    for i in idx:
        if count < valid_ratio * len(files):
            mkdir_if_not_exist([input_dir, 'train', labels['breed'][i]])
            shutil.copy(os.path.join(train_dir, files[i]), os.path.join(input_dir, 'train', labels['breed'][i]))
        else:
            mkdir_if_not_exist([input_dir, 'valid', labels['breed'][i]])
            shutil.copy(os.path.join(train_dir, files[i]), os.path.join(input_dir, 'valid', labels['breed'][i]))
        count += 1
    for file in os.listdir(test_dir):
        mkdir_if_not_exist([input_dir, 'test', 'unknown'])
        shutil.copy(os.path.join(test_dir, file), os.path.join(input_dir, 'test', 'unknown'))

In [4]:
label_file = '/home/samael/kaggle/dogs/labels.csv'
train_dir = '/home/samael/kaggle/dogs/train/'
test_dir = '/home/samael/kaggle/dogs/test/'
input_dir = '/home/samael/kaggle/dogs/data2/'

In [5]:
reorg_data(label_file, train_dir, test_dir, input_dir, valid_ratio=1)

In [6]:
import re
extra_dir = '/home/samael/kaggle/dogs/Images'
for i in tqdm(os.listdir(extra_dir), total=len(os.listdir(extra_dir))):
    directory = re.sub('n[0-9]+-', '', i)
    directory = directory.lower()
    for file in os.listdir(os.path.join(extra_dir, i)):
        shutil.copy(os.path.join(extra_dir, i, file), os.path.join(input_dir, 'train', directory))

100%|██████████| 120/120 [00:32<00:00,  3.72it/s]


In [7]:
preprocessing = [
    image.ForceResizeAug((224,224)),
    image.ColorNormalizeAug(mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225]))
]

def transform(data, label):
    data = data.astype('float32') / 255
    for pre in preprocessing:
        data = pre(data)
    
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')

In [8]:
ctx = mx.gpu(0)
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('/home/samael/kaggle/dogs/data2/train', transform=transform)
data = gluon.data.DataLoader(imgs, 64)

features_vgg, labels = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
features_resnet, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
features_densenet, _ = get_features(models.densenet201(pretrained=True, ctx=ctx), data)

100%|██████████| 482/482 [08:14<00:00,  1.03s/it]


In [9]:
preprocessing[0] = image.ForceResizeAug((299,299))
imgs_299 = vision.ImageFolderDataset('/home/samael/kaggle/dogs/data2/train', transform=transform)
data_299 = gluon.data.DataLoader(imgs_299, 64)

features_inception, _ = get_features(models.inception_v3(pretrained=True, ctx=ctx), data)

100%|██████████| 482/482 [05:04<00:00,  1.58it/s]


In [10]:
with h5py.File('features.h5', 'w') as f:
    f['vgg'] = features_vgg
    f['resnet'] = features_resnet
    f['densenet'] = features_densenet
    f['inception'] = features_inception
    f['labels'] = labels

In [11]:
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('/home/samael/kaggle/dogs/data2/test', transform=transform)
data = gluon.data.DataLoader(imgs, 64)

features_vgg, _ = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
features_resnet, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
features_densenet, _ = get_features(models.densenet201(pretrained=True, ctx=ctx), data)

100%|██████████| 162/162 [02:39<00:00,  1.02it/s]


In [12]:
preprocessing[0] = image.ForceResizeAug((299,299))
imgs_299 = vision.ImageFolderDataset('/home/samael/kaggle/dogs/data2/test', transform=transform)
data_299 = gluon.data.DataLoader(imgs_299, 64)

features_inception, _ = get_features(models.inception_v3(pretrained=True, ctx=ctx), data)

100%|██████████| 162/162 [01:40<00:00,  1.61it/s]


In [13]:
with h5py.File('features_test.h5', 'w') as f:
    f['vgg'] = features_vgg
    f['resnet'] = features_resnet
    f['densenet'] = features_densenet
    f['inception'] = features_inception